In [ ]:
import os 
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # ignore INFO\WARNING

import numpy as np

from make_dir import mkdir

from tensorflow.keras import layers, Sequential

from load_yaml import get_yaml

import models.solutions as solutions
import equations.bgk_eqn as equation

from dataset import Sampler
from solver import Solver

from math import pi
pi = tf.cast(pi, dtype = float)

import matplotlib.pyplot as plt
import time


In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager exacution: {}".format(tf.executing_eagerly()))
# set gpu
gpus = tf.config.list_physical_devices('GPU')
logical_gpus = tf.config.experimental.list_logical_devices(device_type = 'GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")


In [ ]:
# load config
current_path = os.path.abspath(".")
yaml_path = os.path.join(current_path, "kinetic.yaml")
Config = get_yaml(yaml_path)

# load reference data
ref_path = os.path.join(current_path, "data/dirichlet_ref_kn1e0.npz")
ref = np.load(ref_path)
time_freq = 1
ref_rho = ref["density"][::time_freq].astype("float32").reshape(-1, 1) # shape: (100, 1)
ref_momentum = ref["momentum"][::time_freq].astype("float32").reshape(-1, 1)
ref_energy = ref["energy"][::time_freq].astype("float32").reshape(-1, 1)


In [ ]:
# build neural networks for f, rho, u, T
Sol = "solutions.Sol_" + \
    "{}".format(Config["model_config"]["neural_network_type"])
Solution = eval(Sol)

bgk_sol = Solution(units_f=Config["model_config"]["units_f"],
                units_rho=Config["model_config"]["units_rho"],
                units_u=Config["model_config"]["units_u"],
                units_T=Config["model_config"]["units_T"],
                activation=tf.nn.swish,
                kernel_initializer="glorot_normal",
                name="bgk_sol")
bgk_sol.build()

# define the BGK equation
bgk_eqn = equation.BGK(Config, bgk_sol, name="BGK_Eqn")


In [ ]:
instance_t, instance_x, instance_v = tf.split(tf.ones((1, 3)), num_or_size_splits=[1,1,1], axis=-1)
instance_rho, instance_u, instance_T = bgk_sol.macrocall(inputs = tf.concat([instance_t, instance_x], axis=-1))
instance_f = bgk_sol.fcall(inputs = tf.concat([instance_t, instance_x, instance_v], axis=-1))

In [ ]:
weights = np.load("./record/saved_weights.npz", allow_pickle=True)["model_weights"]
bgk_sol.set_weights(weights)


In [ ]:
xmin = Config["dataset_config"]["x_range"][0]
xmax = Config["dataset_config"]["x_range"][1]
tmin = Config["dataset_config"]["t_range"][0]
tmax = Config["dataset_config"]["t_range"][1]

nx = 100
dx = float(xmax - xmin) / nx
ref_x = np.arange(xmin + dx / 2, xmax + dx / 2, dx).reshape((-1, 1))
ref_x = tf.convert_to_tensor(ref_x, dtype=tf.float32)


In [ ]:
def plot_macro(initial=False):
    # initial
    def macro0(macro_l, macro_r, x):
        # y = tf.pow(tf.sin(pi*x*tf.sign(x)), 3 / 7) * tf.sign(x)
        # return macro_l + (macro_r - macro_l) * ((1.0 + y) / 2.0)
        # return macro_l + (macro_r - macro_l) * 0.5 * (tf.tanh(10.0*x) + 1.0)
        return macro_l + (macro_r - macro_l) * 0.5 * (tf.tanh(20.0*x) + 1.0)
            
    if initial:
        # t = 0
        ref_rho = macro0(1.5, 0.625, ref_x)
        ref_u = macro0(0.0, 0.0, ref_x)
        ref_T = macro0(1.5, 0.75, ref_x)
        ref_energy = 0.5 * ref_rho * (ref_u**2 + ref_T)
        ref_momentum = ref_rho * ref_u
        ref_t = np.zeros((nx, 1)) 
    else:
        # t = 0.1
        time_freq = 1
        ref_rho = ref["density"][::time_freq].astype("float32").reshape(-1, 1) # shape: (100, 1)
        ref_momentum = ref["momentum"][::time_freq].astype("float32").reshape(-1, 1)
        ref_energy = ref["energy"][::time_freq].astype("float32").reshape(-1, 1)
        ref_t = tmax * np.ones((nx, 1)) 
    
    ref_t = tf.convert_to_tensor(ref_t, dtype=tf.float32)

    rho_approx, u_approx, T_approx = bgk_sol.macrocall(inputs = tf.concat([ref_t, ref_x], axis=-1))
    approx_density = rho_approx
    approx_momentum = rho_approx * u_approx
    approx_energy = 0.5 * rho_approx * (u_approx**2 + T_approx)

    plt.plot(ref_x, ref_rho, "r", label = "density")
    plt.plot(ref_x, ref_momentum, "k", label = "momentum")
    plt.plot(ref_x, ref_energy, "b", label = "energy")

    plt.plot(ref_x, approx_density, "r*", markevery= 4, label = "approx density")
    plt.plot(ref_x, approx_momentum, "k+", markevery= 4, label = "approx momentum")
    plt.plot(ref_x, approx_energy, "bx", markevery= 4, label = "approx energy")

    plt.grid()
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("macro") # Todo
    plt.title("Approximate density, momentum, energy and reference solutions")
    # plt.savefig("./figure/solution_macro_t5.pdf")
    plt.show()
    plt.close()

In [ ]:
plot_macro(initial=False)
plot_macro(initial=True)

In [ ]:
v, w = np.polynomial.legendre.leggauss(32)
v = 0.5 * (v + 1.0) * 20 - 10
w = 0.5 * 20 * w
vquads = tf.convert_to_tensor(v[:, None], dtype=tf.float32)
wquads = tf.convert_to_tensor(w[:, None], dtype=tf.float32)

In [ ]:
def plot_moments(initial=False):
    # compute moments
    def average_op(func, x, vwquads):
        x = tf.expand_dims(tf.concat(x, axis=-1), axis=-2)
        v, w = vwquads
        xv = tf.concat([x + 0.0 * v[..., 0:1], v + 0.0 * x[..., 0:1]], axis=-1)
        avg = tf.reduce_sum(func(xv) * w, axis=-2)
        return avg

    # initial
    def macro0(macro_l, macro_r, x):
        # y = tf.pow(tf.sin(pi*x*tf.sign(x)), 3 / 7) * tf.sign(x)
        # return macro_l + (macro_r - macro_l) * ((1.0 + y) / 2.0)
        # return macro_l + (macro_r - macro_l) * 0.5 * (tf.tanh(10.0*x) + 1.0)
        return macro_l + (macro_r - macro_l) * 0.5 * (tf.tanh(20.0*x) + 1.0)

    if initial:
        # t = 0
        ref_rho = macro0(1.5, 0.625, ref_x)
        ref_u = macro0(0.0, 0.0, ref_x)
        ref_T = macro0(1.5, 0.75, ref_x)
        ref_energy = 0.5 * ref_rho * (ref_u**2 + ref_T)
        ref_momentum = ref_rho * ref_u
        ref_t = np.zeros((nx, 1)) 
    else:
        # t = 0.1
        time_freq = 1
        ref_rho = ref["density"][::time_freq].astype("float32").reshape(-1, 1) # shape: (100, 1)
        ref_momentum = ref["momentum"][::time_freq].astype("float32").reshape(-1, 1)
        ref_energy = ref["energy"][::time_freq].astype("float32").reshape(-1, 1)
        ref_t = tmax * np.ones((nx, 1))  

    ref_t = tf.convert_to_tensor(ref_t, dtype=tf.float32)

    avg_f_1 = average_op(bgk_sol.fcall, [ref_t, ref_x], [vquads, wquads])
    approx_density = avg_f_1
    avg_f_2 = average_op(bgk_sol.fcall, [ref_t, ref_x], [vquads, wquads * vquads])
    approx_momentum = avg_f_2
    avg_f_3 = average_op(bgk_sol.fcall, [ref_t, ref_x], [vquads, wquads * vquads ** 2])
    approx_energy = 0.5 * avg_f_3

    plt.plot(ref_x, ref_rho, "r", label = "density")
    plt.plot(ref_x, ref_momentum, "k", label = "momentum")
    plt.plot(ref_x, ref_energy, "b", label = "energy")

    plt.plot(ref_x, approx_density, "r*", markevery= 4, label = r"$<f>$")
    plt.plot(ref_x, approx_momentum, "k+", markevery= 4, label = r"$<fv>$")
    plt.plot(ref_x, approx_energy, "bx", markevery= 4, label = r"$\frac{1}{2} <fv^2>$")
    plt.grid()
    plt.legend()
    plt.xlabel("x")
    plt.ylabel("macro") # Todo
    plt.title("Approximate integrals of f and reference solutions")
#     plt.savefig("./figure/solution_f_t1.pdf")
    plt.show()
    plt.close()
    

In [ ]:
plot_moments(initial=False)
plot_moments(initial=True)